In [12]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
import re

from tqdm import tqdm
from bs4 import BeautifulSoup
from utils import *
from row_utils import *

# Set directory & name of save file
results = []
save_file_name = "check1_df"

path_prefix = "./2004/"
all_2004 = os.listdir(path_prefix)
all_2004 = [x for x in all_2004 if ('DS_Store' not in x)]
all_2004 = [x for x in all_2004 if ('xlsx' not in x)]

# Recording which files have been scraped 
cik_done_dict = get_cik_done(save_file_name)
#cik_done_dict = {}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# Scraping the remaining files
results = []
for i in tqdm(range(len(all_2004))):
    
    fname = all_2004[i]
    test_f = open(path_prefix + fname)
    wholetext = test_f.read()
    cikbase = re.search(r"CENTRAL INDEX KEY:\s*?\d{10}\b", wholetext, re.I).group()
    cik_re = re.split(r'\s', cikbase)[-1]        

    datadatebase = re.search(r"CONFORMED PERIOD OF REPORT:\s*?\d{8}\b", wholetext, re.I).group() 
    datadate_re = re.split(r'\s', datadatebase)[-1] 
    if int(cik_re) in cik_done_dict:
        if int(datadate_re) in cik_done_dict[int(cik_re)]:
            print("Already processed this file")
            continue
    
    wholetext = wholetext.lower()
    tbl_flag = 0 # 1 if "contractual obligations" table exists
    row_flag = 0 # 1 if "purchasing obligations" row exists
    multiplier = 1 # 1 by default
    
    table = scrape_table(wholetext)
    rows = []
    if (not table) or ('-----' in table.text):
        multiplier, lines, tbl_list = scrape_text_table(wholetext)
        if tbl_list != None:
            tbl_flag = 1
            rows = scrape_text_rows(lines, tbl_list)
    else:
        tbl_flag = 1
        rows = scrape_rows(table)
        multiplier = get_multiplier_from_tbl(table)
        
    for row in rows:
        row_flag = 1
        result = {'fname': fname, 'cik': cik_re, 'datadate': datadate_re, 'tbl': tbl_flag, 'row': row_flag, 
                   '<1': row['<1'], '1-3':row['1-3'], '3-5':row['3-5'], '>5':row['>5'], 
                  'category': row['category'], 'multiplier': multiplier}
        if 'total' in row.keys():
                result['total'] = row['total']
        results.append(result)

    # No table or no row
    if (tbl_flag == 0) or (row_flag == 0):
        result = {'fname': fname, 'cik': cik_re, 'datadate': datadate_re, 'tbl': tbl_flag, 'row': row_flag, 
                  'total': '-', '<1': '-', '1-3':'-', '3-5':'-', '>5':'-', 'multiplier': multiplier}

        results.append(result)
    pd.DataFrame(results).to_csv(save_file_name)
    print("File: ", i, "\tTable: ", tbl_flag, "Row: ", row_flag)


  0%|          | 0/56 [00:00<?, ?it/s]/Users/divyas/Documents/G1/sec/utils.py:33: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 33 of the file /Users/divyas/Documents/G1/sec/utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  parsed_html = BeautifulSoup(wholetext)
  2%|▏         | 1/56 [00:01<00:56,  1.03s/it]

File:  0 	Table:  1 Row:  0


  4%|▎         | 2/56 [00:02<01:00,  1.12s/it]

File:  1 	Table:  1 Row:  0


  5%|▌         | 3/56 [00:04<01:13,  1.39s/it]

...

File:  53 	Table:  1 Row:  1


 98%|█████████▊| 55/56 [02:06<00:02,  2.20s/it]

File:  54 	Table:  1 Row:  1


100%|██████████| 56/56 [02:08<00:00,  2.12s/it]

File:  55 	Table:  1 Row:  1


In [9]:
#results_df = pd.DataFrame(results)
results_df = pd.read_csv(save_file_name)
row_check = results_df[(results_df['row'] == 0) & (results_df['tbl'] == 1)] # table but no row
none_check = results_df[(results_df['row'] == 0) & (results_df['tbl'] == 0)] # no table no row
last_check = results_df[(results_df['row'] == 1) & (results_df['tbl'] == 1)] # table and row, other issue?
print("row & table found: ", len(set(last_check['fname'])))
print("table found: ", len(set(row_check['fname'])))
print("nothing found: ", len(set(none_check['fname'])))

row & table found:  7
table found:  0
nothing found:  0


In [10]:
last_check

,1-3,3-5,<1,>5,category,cik,datadate,fname,multiplier,row,tbl,total
0,2708.3,1018.7,1719.7,28.0,purchase obligations(a),0000009389,20031231,0000009389-04-000059.txt,1000000,1,1,5474.7
1,3204.9,1128.3,1832.1,1113.1,total payments on contractual obligations,0000009389,20031231,0000009389-04-000059.txt,1000000,1,1,7278.4
2,1,0,98,0,and construction of buildings,0000003333,20040129,0000003333-04-000014.txt,1,1,1,99.0
3,0,0,80,0,transportation contracts,0000003333,20040129,0000003333-04-000014.txt,1,1,1,80.0
4,1346,831,1480,7532,total contractual cash obligations,0000003333,20040129,0000003333-04-000014.txt,1,1,1,11189.0
5,77,49,63,31,capital lease obligations,0000022198,20031231,0000004904-04-000060.txt,1000000,1,1,220.0
6,2132,1101,1720,1785,obligations (a),0000022198,20031231,0000004904-04-000060.txt,1000000,1,1,6738.0
7,492,441,291,2331,noncancellable operating leases,0000022198,20031231,0000004904-04-000060.txt,1000000,1,1,3555.0
8,2512,0,2512,7200,other long0term obligations (b),0000800459,20040630,0000800459-04-000047.txt,1000,1,1,12224.0
9,0,0,20674,000,firm commitments for capital expenditures,0000800459,20040630,0000800459-04-000047.txt,1000,1,1,20674.0


# Examining one file

In [9]:
test_f = open(path_prefix + "0000733269-04-000010.txt")
wholetext = test_f.read()
cik_re = get_cik(wholetext)
datadate_re = get_datadate(wholetext)  
print("CIK: ", cik_re, "Date: ", datadate_re)
wholetext = wholetext.lower()
multiplier, lines, tbl_list = scrape_text_table(wholetext)
rows = scrape_text_rows(lines, tbl_list)
#scores, tables = scrape_table(wholetext)
#table = scrape_table(wholetext)
#rows = scrape_rows(table)
#lines
#mult = get_multiplier_from_tbl(table)
rows

CIK:  0000733269 Date:  20040331


[{'1-3': 24309.0,
  '3-5': 4730.0,
  '<1': 38588.0,
  '>5': '11821',
  'category': 'capital lease obligations',
  'total': 79448.0},
 {'1-3': 75606.0,
  '3-5': 205824.0,
  '<1': 73246.0,
  '>5': '12026',
  'category': 'obligations',
  'total': 366702.0},
 {'1-3': 6044.0,
  '3-5': 6044.0,
  '<1': 3022.0,
  '>5': '11389',
  'category': 'synthetic aircraft leases',
  'total': 26499.0},
 {'1-3': 14428.0,
  '3-5': 715.0,
  '<1': 20401.0,
  '>5': '0',
  'category': 'equipment operating leases',
  'total': 35544.0},
 {'1-3': 29792.0,
  '3-5': 23301.0,
  '<1': 16831.0,
  '>5': '59430',
  'category': 'building operating leases',
  'total': 129354.0},
 {'1-3': 4257.0,
  '3-5': 4299.0,
  '<1': 2122.0,
  '>5': '43',
  'category': 'partnerships building leases',
  'total': 10721.0},
 {'1-3': 1278.0,
  '3-5': 0.0,
  '<1': 903.0,
  '>5': '0',
  'category': 'related party aircraft lease',
  'total': 2181.0},
 {'1-3': 8556.0,
  '3-5': 7030.0,
  '<1': 9426.0,
  '>5': '3515',
  'category': 'obligations',

In [293]:
#lines = chunks[np.argmax(scores)].split('\n')
lines = [x for x in lines if (hasNumbers(x))]
keywords = HDR_KWS + ROW_KWS + ['obligations']
lines = [x for x in lines if (hasKeywords(x, keywords))]
lines = [x.split(' ') for x in lines]
for i, line in enumerate(lines):
    lines[i] = list(filter(lambda a: (a != '') and (a != '$') and (a !='<u>') and (a !='</u>'), line))    
tbl_list = []
tbl_list = combine_text_fields(lines)
scrape_text_rows(lines, tbl_list)

Unknown error:  could not convert string to float: 'by inflation of'


[{'1-3': 2848000.0,
  '3-5': '71000',
  '<1': 1179000.0,
  '>5': '000',
  'category': 'obligations',
  'total': 4098000.0},
 {'1-3': 24293.0,
  '3-5': '0',
  '<1': 24293.0,
  '>5': '0',
  'category': 'obligations',
  'total': 48586.0},
 {'1-3': 0.0,
  '3-5': '0',
  '<1': 3450.0,
  '>5': '0',
  'category': 'purchase obligations',
  'total': 3450.0}]

In [135]:
#lt[1].text.replace('\n', '').replace('\xa0', '')
len(lt)

1